# Quirks of Jupyter notebooks

This notebook demonstrates some behaviours specific to IPython and Jupyter notebooks that we encounter in Qiskit and its docs.

## 1. Jupyter `display`

Key points:
* IPython (including Jupyter) has a built-in function named `display`. This is **not** a standard Python built-in.
* IPython evaluates and _`display`s_ the last line of a code cell.

For example, the last line of the following cell evaluates to the string `'Hello, world!'`.

In [ ]:
x = "Hello, world!"
x

Note the quotes around the string, this is because `display` shows the `__repr__` method of an object, whereas `print` prints the `__str__` method.

In [ ]:
print(x)
print(x.__repr__())
display(x)  # note no import needed
x  # `display` is called on this value to produce the cell output

If an object doesn't have a `__repr__`, `display` will show the object type and pointer (this is usually useless to users).

In [ ]:
class Frank:
    pass


Frank()

Authors can define their own `__repr__` methods. This method _should_ return the Python string you'd evaluate to create that object (that's why the quotes were included in the previous cell outputs). However, this isn't always possible, so some objects return an approximation or omit information (for an example, try displaying a large numpy array).

In [ ]:
class FrankWithRepr:
    def __repr__(self):
        return "FrankWithRepr(...)"


FrankWithRepr()

This can cause confusion with users coming from Python when they copy code from a notebook and see no output in their terminal. The solution is usually to tell them to `print` the appropriate lines.

## 2. Rich outputs

One of the main advantages to running IPython in a browser is the ability to display images and other rich content inline. Jupyter does this using special display methods. For example, the following object has a method named `_repr_svg_`; `display` recognizes this method and calls it, then shows the output as an SVG to the user.

Note this object also has a `__repr__` method; Jupyter does not display the `__repr__` to the user since there is a richer alternative (SVG) available. However, it _does_ call the `__repr__` method and saves the output in the notebook (check the notebook source to see). This is so different viewers can choose how they want to display each notebook. For a full list of rich outputs, see https://ipython.readthedocs.io/en/stable/config/integrating.html.

In [ ]:
class Circle:
    def __init__(self, color: str):
        self.color = color

    def __repr__(self):
        return f"Circle(color={self.color.__repr__()})"

    def _repr_svg_(self):
        return f"""
          <svg version="1.1" width="300" height="200" xmlns="http://www.w3.org/2000/svg">
            <circle cx="150" cy="100" r="80" fill="{self.color}" />
          </svg>
          """


Circle(color="red")

> **Note:** A very relevant example in Qiskit is `QuantumCircuit.draw`, which returns a Matplotlib `Figure` object. The matplotlib object has a rich output, which is why it displays an image when it's the last line of a cell. Users are often confused as to why `qc.draw()` doesn't "do anything" when called in a loop, the solution is usually to replace the line with `display(qc.draw())`.

## 3. Input/Output caching system

Jupyter saves the inputs and results of past cell executions in memory. I recommend **not** using this in user-facing code as it can quickly become confusing, but it can be very useful for testing.

In [ ]:
# Underscore (_) stores the output of the most recently executed code cell.
# In this case, the Circle object.
_

In [ ]:
# Can use this to check properties of cell outputs
assert _.color == 'red'

In [ ]:
print(_i) # Input of previous execution

For more information, see docs for the [input caching system](https://ipython.readthedocs.io/en/stable/interactive/reference.html#input-caching-system) and the [output caching system](https://ipython.readthedocs.io/en/stable/interactive/reference.html#output-caching-system).

## 4. Magic commands

Jupyter also supports "magic" commands that start with `%` (for a single-line command), or `%%` (if the command should apply to the whole cell). We won't go into this much and I also recommend avoiding these in user-facing code. The only magic we really use is the `%%writefile` magic, which we use to write a cell to disk rather than executing it. We have some display logic in inner-source that detects the `%%writefile` magic and displays the cell differently to the user. See https://quantum.cloud.ibm.com/docs/en/guides/serverless-manage-resources#set-detailed-statuses for an example. 

For more information on magics in general, see https://ipython.readthedocs.io/en/stable/interactive/reference.html#magic-command-system.

In [ ]:
%%writefile example.py
# This cell will not execute, it will be written to `example.py` instead
print('Hello, world!')